In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

df = pd.read_csv('./data/cabaventa.csv')
df.shape

## Limpieza Texto y Columnas

En esta parte se realiza una limpieza básica con `regex` de algunas columnas a los fines de normalizar un poco el texto: lowering, punctuation, spaces, digits. La función armada permite customizar la limpieza, por ende es posible no aplicar la misma limpieza para todos los casos. La función genera columnas nuevas con el nombre`f"{col}_cleaned"`  

Dado que las columnas textuales `title` y `description` no fueron utilizadas en la predicción ni analizadas mediante técnicas de embeddings o BOW, no se trataron los NaN en esas columnas. 

In [ ]:
from cleaning import CleaningData
#instanciamos Cleaning Data con df

cleaned = CleaningData(data=df)

In [ ]:
#Quitamos columnas 
cols_to_drop = ['Unnamed: 0.1', 'Unnamed: 0', #columnas que vinieron mal en la bajada
                'operation_type', #todos son 'venta'
                'l1', #todas las filas iguales 
                'l2', #todas las filas iguales 
                'ad_type', #todas las filas iguales 
                'l4', #todas las filas nulas preferible el dato de lat y long
                'l5', #columna con todas las filas nulas
                'l6', #columna con todas las filas nulas
                'created_on', #la columna created_on es igual a la columna start_date
                'price_period' #la columna contiene muchos NaN y un único valor
               ]
data = cleaned.drop_columns(columns=cols_to_drop)
data.shape

In [ ]:
#Limpieza title
params = {'lowering':True,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="title", params=params)
data.shape

In [ ]:
#Limpieza description
params = {'lowering':True,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="description", params=params)
data.shape

In [ ]:
#Limpieza texto de l3
params = {'lowering':False,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="l3", params=params)
data.shape

## Barrio, latitud y longitud

#### Valores nulos

In [ ]:
(data[['lat','lon','l3_cleaned']].isna().sum()/len(data)).apply(lambda x: "{:.2%}".format(x))

Se detectó una mayor presencia de valores nulos en los datos de georreferenciación que en el dato del barrio. Por ende se realizó un trabajo de imputación de nulos partiendo de considerar a la columna `l3_cleaned` con una mayor validez que la columnas `lat` y `lon`. Por ende, fue tomada como "eje" para realizar imputación de nulos. 

#### Errores

* Barrios cuya denomiación no es la denominación oficial: Ejemplo Centro / Microcentro, Las Cañitas, Pompeya, Abasto, Once, etc. 

In [ ]:
data.l3_cleaned.unique()

* Columna `lat` con valores fuera del polígono de CABA

In [ ]:
data.lat.describe()

* Columna `lon` con valores fuera de los valores correctos. 

In [ ]:
data.lon.describe()

- Observo las propiedades que "caen" fuera de CABA. Reemplazo por nulo aquellas propiedades cuyas coordenadas no tienen sentido 

In [ ]:
sns.scatterplot(data.lon,data.lat)
plt.show()

In [ ]:
index_lat_lon = data[(data['lat'] > -34.5) | (data['lon'] < -58.55) | (data['lat'] < -34.685) | (data['lon'] > -58)].index.values
data.loc[index_lat_lon, ['lat', 'lon']] = np.nan

In [ ]:
sns.scatterplot(data.lon,data.lat)
plt.show()

Creo una variable para completar con 1 en aquellas instancias que voy a eliminar

In [ ]:
data['to_be_erased'] = 0

Considerando la importancia de la locación en la predicción de un precio de una propiedad y la complejidad de extraer el dato de la descripción, se vna a eliminar aquellas sin barrio ni coordenadas geográficas

In [ ]:
index_no_location = data[(data['lat'].isna()) & (data['l3_cleaned'].isna())].index.values
data.loc[index_no_location, 'to_be_erased'] = 1

Para aquellas propiedades que sí tienen el barrio, pero no las coordenadas geográficas, voy a completar con el promedio de las coordenadas geográficas del barrio.

In [ ]:
lat_lon_barrios = data.groupby('l3_cleaned')[['lat', 'lon']].mean()
lat_lon_barrios.columns = ['lat_barrio', 'lon_barrio']

data = pd.merge(data, lat_lon_barrios, how="left", left_on='l3_cleaned', right_on = 'l3_cleaned')

data['lat'] = np.where(data['lat'].isna(), data['lat_barrio'], data['lat'])
data['lon'] = np.where(data['lon'].isna(), data['lon_barrio'], data['lon'])

Por el momento, borro aquellas propiedades que no tienen coordenadas geográficas. En todo caso, se podría completar utilizando los polígonos de cada barrio y viendo en cual cae la propiedad

In [ ]:
index_no_barrio = data[data['l3_cleaned'].isna()].index.values
data.loc[index_no_barrio, 'to_be_erased'] = 1

In [ ]:
data = data.drop(columns = ['lat_barrio', 'lon_barrio'])

#### Procesamiento realizado

Se establecieron los siguientes pasos: 
1) Normalización datos barrios: dado que eran pocos casos se realizó una imputación manual sobre la columna `l3_cleaned`. Ej: Las Canitas ---> Palermo. Pompeya --> Nueva Pompeya. Abasto --> Almagro.
2) Si la latitud, longitud fueron NaN o localización fuera de CABA se completó con el promedio de las coordenadas del barrio, si para dicho registro el dato del barrio fue nulo entonces se quitó toda la fila. 
3) Si el barrio fue nulo pero la localización dentro de CABA, se completó barrio verificando en qué polígono pertencía el punto.
4) Si el barrio y la localización no son nulos pero sí incongruentes (localización pertence a otro barrio) se procedío a imputar el punto centroide del barrio. 

Pendiente para un próximo abordaje: 

* Para los casos en que la localización era externa a CABA, el barrio no nulo, se imputó un punto centroide del barrio. Eso pudo conducir a errores dado que implícitamente se "confía" en que el dato del barrio fue correcto. Una alternativa sería probar adicionalmente si de la columna `description_cleaned` fuera posible confirmar que no se tratase de casos que efectivamente pertenecieran a localizaciones externas a CABA. 

## Superficie total, superficie cubierta, ambientes, dormitorios y precio

#### Valores nulos

In [ ]:
cols = ["surface_total","surface_covered","rooms","bedrooms", "price"]
(data[cols].isna().sum()/len(data)).apply(lambda x: "{:.2%}".format(x))

Dada la alta correlación entre `surface_total` y `surface_covered` y entre `rooms` y `bedrooms`, para los casos en que el par tuviera datos nulos se procedió a eliminar la fila ante la dificultad para imputar nulos en esos casos. 

```
data.dropna(subset = ['rooms', bedrooms'])
data.dropna(subset = ['surface_total', 'surface_covered'])
```

Para la variable `price` se eliminaron las filas que tenían nulos ya que por ser la variable sobre la cual se constuirá la variable respuesta se decidió no realizar imputación de valores faltantes sobre la misma. 

```
data.dropna(subset =['price'])
```

#### Errores / Outliers

Se detecton que las columnas contenían errores o valores outliers. Se enumeran a continuación: 

* `surface_total < surface_covered`

In [ ]:
(data.surface_total < data.surface_covered).sum() 

* outlieres en `surface_total`

In [ ]:
data.surface_total.describe()

* outliers en `surface_covered`

In [ ]:
data.surface_covered.describe()

* `rooms < bedrooms`

In [ ]:
(data.rooms < data.bedrooms).sum() 

* outliers en `rooms`

In [ ]:
data.rooms.describe()

* outliers en `bedrooms`

In [ ]:
data.bedrooms.describe()

#### Procesamiento realizado

1) Borrar filas donde los pares de columnas `['rooms','bedrooms']` o `['surface_total','surface_covered']` contengan nulos. 
2) Intercambiar los valores de las columnas cuando `rooms < bedrooms` o `surface_total < surface_covered`. 
3) Eliminar filas donde `price` sea nulo. 
4) Si para una fila uno de los valores de `['surface_total', 'surface_covered']` contiene valor outlier y el otro no es outlier, se le imputó un `NaN` al dato outlier y se dejó dicho valor para ser imputado luego por método MICE. En caso que los dos valores fueran outliers o uno outlier y el otro `NaN` se borró la fila. 
5) Si para una fila uno de los valores de `['rooms', bedrooms']` contiene valor outlier y el otro no es outlier, se le imputó un `NaN` al dato outlier y se dejó dicho valor para ser imputado luego por método MICE. En caso que los dos valores fueran outliers o uno outlier y el otro `NaN` se borró la fila. 
6) Se borraron los valores outliers de `price`
7) Luego de estos pasos se procedió a imputar los valores nulos con el método multivariado MICE en las columnas `['rooms', bedrooms','surface_total','surface_covered']` utilizando también la variable `price` como predictor. 

## Modificación de los tipos de datos

Acá hay que señalar si se realizaron cambios en los tipos de datos: ejemplo todas las variables que se pasan a categoricas para mejorar el uso de memoria que realiza pandas. 

## Resultados Pre-procesamiento. 

Enumerar cantidad de filas y columnas iniciales, enumerar cantidad de filas y columnas resultantes. 